# Prise en main de PySpark

## Installation et configuration

**Question 1 : pré-requis**

Téléchargez le fichier `cereal.csv` sur CURSUS et ajoutez-le dans un nouveau dossier `/user/VOTRE_PRENOM/tp_pyspark` sur le HDFS.

**Question 2**

Exécutez la cellule ci-dessous pour connaître le nombre de coeurs de calcul qui vous sont attribués sur la machine actuelle.

In [ ]:
from os import cpu_count
# get the number of logical cpu cores
n_cores = cpu_count()
# report the number of logical cpu cores
print(f'Number of Logical CPU cores: {n_cores}')

Number of Logical CPU cores: 2


**Question 3**

Ci-dessous, créer une `SparkSession` pour une application nommée "Test de PySpark" et lancée en local avec le nombre de coeurs trouvé ci-dessus.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Test de PySpark").master("yarn").getOrCreate()

## Quelques tests sur les RDDs

### Transformations, actions et persistence

**Question 4**

Ci-dessous, créer deux RDDs (nommés `my_rdd1`et `my_rdd2`) à partir de deux listes Python de 6 éléments chacune.

Afficher ces `RDD` avec `print()` ainsi que leur nombre de partitions.

In [ ]:
sc = spark.sparkContext

my_rdd1 = sc.parallelize(list(range(0,6)))
my_rdd2 = sc.parallelize(list(range(6,12)))

print(my_rdd1)
print(my_rdd2)
print(my_rdd1.getNumPartitions())
print(my_rdd2.getNumPartitions())

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289
ParallelCollectionRDD[1] at readRDDFromFile at PythonRDD.scala:289
2
2


In [ ]:
import time
from operator import add

def carre_slow(x):
  time.sleep(1)
  return x**2


**Question 5**

Executer la cellule de code ci-dessus, puis utiliser la méthode `map`des RDDs pour générer un nouveau RDD nommé `my_new_rdd1` contenant les carrés des valeurs de `my_rdd1`, en utilisant la fonction `carre_slow` définie ci-dessus.

Mesurer le temps d'exécution et conclure : les traitements ont-ils réellement été exécutés à cet endroit ?

Afficher le nouveau RDD généré et son nombre de partitions.

In [ ]:
%%time
my_new_rdd1 = my_rdd1.map(carre_slow)
print(my_new_rdd1)
print(my_new_rdd1.getNumPartitions())

PythonRDD[2] at RDD at PythonRDD.scala:53
2
CPU times: user 4.43 ms, sys: 1.27 ms, total: 5.7 ms
Wall time: 25.3 ms


**Question 6**

En mesurant le temps d'exécution, utiliser la méthode `collect` pour récupérer et afficher le contenu du RDD `my_new_rdd1` dans le *driver program* (programme principal, ici celui du Notebook).

Conclure quant aux traitements lancés au moment de l'appel à `collect` et à la nature de cette méthode : est-ce une transformation ou une action ?

In [ ]:
%%time
my_new_rdd1.collect()

CPU times: user 27.9 ms, sys: 5.93 ms, total: 33.9 ms
Wall time: 5.12 s


[0, 1, 4, 9, 16, 25]

**Question 7**

En mesurant les temps d'exécution, calculer à l'aide de la méthode `reduce`la somme des carrés contenus dans `my_new_rdd1` (on pourra utiliser la fonction `add` du module `operator` importée ci-dessus).

Qu'observez-vous au niveau du temps d'exécution ?

In [ ]:
%%time
my_new_rdd1.reduce(add)

CPU times: user 25.4 ms, sys: 4.89 ms, total: 30.3 ms
Wall time: 3.77 s


55

**Question 8**

De la même manière, générer un nouveau RDD nommé `my_new_rdd2` contenant les carrés des valeurs de `my_rdd2`, en utilisant la fonction `carre_slow` puis appeler la méthode `persist` sur ce nouveau RDD.

Dans la seconde cellule, en mesurant le temps d'exécution, afficher le contenu de `my_new_rdd2` en utilisant la méthode `collect`.

In [ ]:
my_new_rdd2 = my_rdd2.map(carre_slow)
my_new_rdd2.persist()


PythonRDD[4] at RDD at PythonRDD.scala:53

In [ ]:
%%time
my_new_rdd2.collect()

CPU times: user 25.6 ms, sys: 1.51 ms, total: 27.1 ms
Wall time: 4.11 s


[36, 49, 64, 81, 100, 121]

**Question 9**

En mesurant les temps d'exécution, calculer à l'aide de la méthode `reduce`la somme des carrés contenus dans `my_new_rdd2`.

Qu'observez-vous cette fois-ci au niveau du temps d'exécution ?

Conclure : à votre avis, à quoi sert la méthode `persist` appelée ci-dessus sur `my_new_rdd2` ? Vérifier en consultant la [documentation en ligne](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.persist.html#pyspark.RDD.persist).

In [ ]:
%%time
my_new_rdd2.reduce(add)

CPU times: user 7.7 ms, sys: 650 µs, total: 8.35 ms
Wall time: 337 ms


451

## Utilisation de Spark SQL

### Lecture et mise en forme des données

*Remarque : lors des différentes transformations de votre DataFrame il est conseillé d'afficher le résultat de vos traitements avant de remplacer la DataFrame*

**Question 10**

Charger le fichier `cereal.csv` dans une DataFrame Spark SQL nommée `sdf1` (une description des données peut être trouvée [ici](https://www.kaggle.com/datasets/crawford/80-cereals/)).

Vérifier que vous récupérer bien les en-têtes de colonne. Dans le cas contraire, adapter l'appel utilisé pour la lecture des données du fichier.

Afficher le nombre de lignes contenus dans cette DataFrame, son nombre de partitions (en passant par le RDD sous-jacent) et ses 50 premières lignes.

In [ ]:
sdf1 = spark.read.csv("hdfs://dataproc-cluster-m/user/ROMAIN/tp_pyspark/cereal.csv", header=True, inferSchema=True)
print(sdf1.count())
print(sdf1.rdd.getNumPartitions())
sdf1.show(50)

77
1
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
|Apple Cinn

**Question 11**

Afficher le schéma de la DataFrame et vérifier que vous obtenez les bons types. Dans le cas contraire, adapter l'appel utilisé pour la lecture des données du fichier afin d'inférer automatiquement le schéma.

In [ ]:
sdf1.printSchema()

root
 |-- name: string (nullable = true)
 |-- mfr: string (nullable = true)
 |-- type: string (nullable = true)
 |-- calories: integer (nullable = true)
 |-- protein: integer (nullable = true)
 |-- fat: integer (nullable = true)
 |-- sodium: integer (nullable = true)
 |-- fiber: double (nullable = true)
 |-- carbo: double (nullable = true)
 |-- sugars: integer (nullable = true)
 |-- potass: integer (nullable = true)
 |-- vitamins: integer (nullable = true)
 |-- shelf: integer (nullable = true)
 |-- weight: double (nullable = true)
 |-- cups: double (nullable = true)
 |-- rating: double (nullable = true)



**Question 12**

En une seule instruction, supprimer la colonne intitulée `"shelf"` puis renommer l'ensemble des colonnes restant (en utilisant une des méthodes `toDF` ou `withColumnRenamed`) avec, dans l'ordre, les intitulés suivants :

```python
"Nom", "Fabricant","Type", "Calories","Protéines", "Graisse","Sodium","Fibres",\
"Glucides","Sucre","Potassium", "Vitamines", "Poids","Volume","Evaluation"
```

Afficher la nouvelle DataFrame obtenue.

In [ ]:
sdf1 = sdf1.drop("shelf").toDF("Nom", "Fabricant","Type", "Calories","Protéines",
                 "Graisse","Sodium","Fibres","Glucides","Sucre",
                 "Potassium", "Vitamines", "Poids","Volume","Evaluation")
sdf1.show()

+--------------------+---------+----+--------+---------+-------+------+------+--------+-----+---------+---------+-----+------+----------+
|                 Nom|Fabricant|Type|Calories|Protéines|Graisse|Sodium|Fibres|Glucides|Sucre|Potassium|Vitamines|Poids|Volume|Evaluation|
+--------------------+---------+----+--------+---------+-------+------+------+--------+-----+---------+---------+-----+------+----------+
|           100% Bran|        N|   C|      70|        4|      1|   130|  10.0|     5.0|    6|      280|       25|  1.0|  0.33| 68.402973|
|   100% Natural Bran|        Q|   C|     120|        3|      5|    15|   2.0|     8.0|    8|      135|        0|  1.0|   1.0| 33.983679|
|            All-Bran|        K|   C|      70|        4|      1|   260|   9.0|     7.0|    5|      320|       25|  1.0|  0.33| 59.425505|
|All-Bran with Ext...|        K|   C|      50|        4|      0|   140|  14.0|     8.0|    0|      330|       25|  1.0|   0.5| 93.704912|
|      Almond Delight|        R|  

**Question 13**

En utilisant les dictionnaires `dic_fabricant` et `dic_type` ci-dessous, remplacer à l'aide de la méthode `replace` les valeurs des colonnes `Fabricant` et `Type` par les nouvelles valeurs associées.

In [ ]:
dic_fabricant = {"A":"American Home Food Products",
    "G" : "General Mills",
    "K" : "Kelloggs",
    "N" : "Nabisco",
    "P" : "Post",
    "Q" : "Quaker Oats",
    "R" : "Ralston Purina"}

dic_type = {"C" : "froid", "H" : "chaud"}

sdf1 = sdf1.replace(dic_fabricant, subset=["Fabricant"]).replace(dic_type, subset =["Type"])

sdf1.show(50)


+--------------------+--------------------+-----+--------+---------+-------+------+------+--------+-----+---------+---------+-----+------+----------+
|                 Nom|           Fabricant| Type|Calories|Protéines|Graisse|Sodium|Fibres|Glucides|Sucre|Potassium|Vitamines|Poids|Volume|Evaluation|
+--------------------+--------------------+-----+--------+---------+-------+------+------+--------+-----+---------+---------+-----+------+----------+
|           100% Bran|             Nabisco|froid|      70|        4|      1|   130|  10.0|     5.0|    6|      280|       25|  1.0|  0.33| 68.402973|
|   100% Natural Bran|         Quaker Oats|froid|     120|        3|      5|    15|   2.0|     8.0|    8|      135|        0|  1.0|   1.0| 33.983679|
|            All-Bran|            Kelloggs|froid|      70|        4|      1|   260|   9.0|     7.0|    5|      320|       25|  1.0|  0.33| 59.425505|
|All-Bran with Ext...|            Kelloggs|froid|      50|        4|      0|   140|  14.0|     8.0| 

**Question 14**

En utilisant la méthode `withColumns`, convertir les poids donnés en *ounces* (onces) en grammes (1 once = 28,35 grammes) et les volumes donnés en *cups* (tasses) en litres (1 tasse = 0,25 litres).

In [ ]:
sdf1.withColumns({"Poids": sdf1.Poids*28.35, "Volume": sdf1.Volume*0.25}).show()

+--------------------+--------------+-----+--------+---------+-------+------+------+--------+-----+---------+---------+-------+------+----------+
|                 Nom|     Fabricant| Type|Calories|Protéines|Graisse|Sodium|Fibres|Glucides|Sucre|Potassium|Vitamines|  Poids|Volume|Evaluation|
+--------------------+--------------+-----+--------+---------+-------+------+------+--------+-----+---------+---------+-------+------+----------+
|           100% Bran|       Nabisco|froid|      70|        4|      1|   130|  10.0|     5.0|    6|      280|       25|  28.35|0.0825| 68.402973|
|   100% Natural Bran|   Quaker Oats|froid|     120|        3|      5|    15|   2.0|     8.0|    8|      135|        0|  28.35|  0.25| 33.983679|
|            All-Bran|      Kelloggs|froid|      70|        4|      1|   260|   9.0|     7.0|    5|      320|       25|  28.35|0.0825| 59.425505|
|All-Bran with Ext...|      Kelloggs|froid|      50|        4|      0|   140|  14.0|     8.0|    0|      330|       25|  28.

### Re-partitionnement et écriture des nouvelles données

**Question 15**

A l'aide de la méthode `repartitionByRange` re-partitionner la DataFrame `sdf1` en 2 partitions en fonction du fabricant.

Afficher le nombre de partitions de la nouvelles DataFrame `sdf1` et ses 50 premières lignes. Comparer à l'affichage obtenu à la question précdente.

In [ ]:
sdf1 = sdf1.repartitionByRange(2, "Fabricant")
print(sdf1.rdd.getNumPartitions())
sdf1.show(50)

2
+--------------------+--------------------+-----+--------+---------+-------+------+------+--------+-----+---------+---------+-----+------+----------+
|                 Nom|           Fabricant| Type|Calories|Protéines|Graisse|Sodium|Fibres|Glucides|Sucre|Potassium|Vitamines|Poids|Volume|Evaluation|
+--------------------+--------------------+-----+--------+---------+-------+------+------+--------+-----+---------+---------+-----+------+----------+
|            All-Bran|            Kelloggs|froid|      70|        4|      1|   260|   9.0|     7.0|    5|      320|       25|  1.0|  0.33| 59.425505|
|All-Bran with Ext...|            Kelloggs|froid|      50|        4|      0|   140|  14.0|     8.0|    0|      330|       25|  1.0|   0.5| 93.704912|
|Apple Cinnamon Ch...|       General Mills|froid|     110|        2|      2|   180|   1.5|    10.5|   10|       70|       25|  1.0|  0.75| 29.509541|
|         Apple Jacks|            Kelloggs|froid|     110|        2|      0|   125|   1.0|    11.0

**Question 16**

Ecrire la nouvelle Dataframe re-partitionnée `sdf1` dans un sous-répertoire `cereal_french` du répertoire `tp_pyspark` du HDFS au format `csv` avec les en-têtes. Observer le résutat obtenu.

D'après le contenu de chaque fichier obtenu, comment ont été faites les partitions de `sdf1` ?

In [ ]:
sdf1.write.csv("hdfs://dataproc-cluster-m/user/ROMAIN/tp_pyspark/cereal_french", header=True)

### Filtrage, tri, groupements et agrégation.

**Question 17**

Afficher les lignes correspondant aux céréales avec des glucides strictement inférieurs à 10 grammes (on excluera également les valeurs négatives de glucides qui correspondent à des informations non renseignées). Ordonner les résultats suivant les glucides.

In [ ]:
sdf1.filter((sdf1.Glucides < 10) & (sdf1.Glucides >= 0)).orderBy(sdf1.Glucides).show()

+--------------------+-----------+-----+--------+---------+-------+------+------+--------+-----+---------+---------+-----+------+----------+
|                 Nom|  Fabricant| Type|Calories|Protéines|Graisse|Sodium|Fibres|Glucides|Sucre|Potassium|Vitamines|Poids|Volume|Evaluation|
+--------------------+-----------+-----+--------+---------+-------+------+------+--------+-----+---------+---------+-----+------+----------+
|           100% Bran|    Nabisco|froid|      70|        4|      1|   130|  10.0|     5.0|    6|      280|       25|  1.0|  0.33| 68.402973|
|            All-Bran|   Kelloggs|froid|      70|        4|      1|   260|   9.0|     7.0|    5|      320|       25|  1.0|  0.33| 59.425505|
|   100% Natural Bran|Quaker Oats|froid|     120|        3|      5|    15|   2.0|     8.0|    8|      135|        0|  1.0|   1.0| 33.983679|
|All-Bran with Ext...|   Kelloggs|froid|      50|        4|      0|   140|  14.0|     8.0|    0|      330|       25|  1.0|   0.5| 93.704912|
|            

**Question 18**

Afficher le nombre de céréales par type (chaud/froid).

In [ ]:
sdf1.groupBy("Type").count().show()

+-----+-----+
| Type|count|
+-----+-----+
|chaud|    3|
|froid|   74|
+-----+-----+



**Question 19**

Créer une nouvelle DataFrame `sdf2` contenant les moyennes des grandeurs numériques par fabricant et renommer les colonnes de cette DataFrame avec dans l'ordre :
```
"Fabricant","Calories","Protéines","Graisse","Sodium","Fibres",
"Glucides","Sucre","Potassium", "Vitamines", "Poids","Volume","Evaluation"
```

Afficher le nombre de partitions de cette DataFrame `sdf2` et son contenu.

In [ ]:
sdf2 = sdf1.groupBy("Fabricant").mean().toDF("Fabricant","Calories","Protéines",
                 "Graisse","Sodium","Fibres","Glucides","Sucre",
                 "Potassium", "Vitamines", "Poids","Volume","Evaluation")
print(sdf2.rdd.getNumPartitions())
sdf2.show()

2
+--------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+
|           Fabricant|          Calories|         Protéines|            Graisse|            Sodium|            Fibres|          Glucides|             Sucre|         Potassium|        Vitamines|             Poids|            Volume|       Evaluation|
+--------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+
|            Kelloggs|108.69565217391305| 2.652173913043478| 0.6086956521739131| 174.7826086956522| 2.739130434782609|15.130434782608695| 7.565217391304348|103.04347826086956|34.78260869565217|1.0778260869565217|0.7960869565217392|44.03846234782609

**Question 20**

En utilisant la méthode `coalesce` rassembler la DataFrame `sdf2` en une seule partition puis l'écrire dans un répertoire `./fabricant_valeurs_moyennes` au format `csv` (avec les en-têtes). Observer le résutat obtenu.

In [ ]:
sdf2.coalesce(1).write.csv("hdfs://dataproc-cluster-m/user/ROMAIN/tp_pyspark/fabricant_valeurs_moyennes", header=True)

**Question 21**

Exécuter la cellule de code ci-dessous pour arrêter votre `SparkSession` (on suppose que votre objet `SparkSession` s'appelle `spark`).

In [ ]:
spark.stop()